In [1]:
import pickle
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from torch.utils.data import DataLoader, Dataset

In [2]:
# File path
FILE_PATH = './Dataset/EURUSD/EURUSD_M30_features+label_v.2.1.csv'
PICKLE_LOAD_PATH = r"D:/Programing/AI Trader/Dataset/EURUSD/EURUSD_M30_features+label_v.2.2.pkl"

In [3]:
# Hyperparameters
FILE_PATH = './Dataset/EURUSD/EURUSD_M30_features+label_v.2.1.csv'
SEQ_LENGTH = 15
NUM_LAYERS = 2
NUM_HEADS = 4
BATCH_SIZE = 1024
EPOCHS = 50
DROPOUT = 0.4
LEARNING_RATE = 0.005
input_dim = 4

In [4]:
with open(PICKLE_LOAD_PATH, 'rb') as file:
    data = pickle.load(file)

In [5]:
# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
# Read and preprocess data
data = pd.read_csv(FILE_PATH)
data = data[['Close', 'SMA200', 'SMA50', 'RSI14', 'signal']]
data[['Close', 'SMA200', 'SMA50', 'RSI14']] = MinMaxScaler().fit_transform(data[['Close', 'SMA200', 'SMA50', 'RSI14']])
data = data.round(4)
data['signal'] = data['signal'].apply(lambda x: x if x in [1, 2, 3] else 3)

In [6]:
# Dataset preparation
class ForexDataset(Dataset):
    def __init__(self, data, seq_length):
        self.features = data[['Close', 'SMA200', 'SMA50', 'RSI14']].values
        self.labels = data['signal'].values
        self.seq_length = seq_length
    
    def __len__(self):
        return len(self.features) - self.seq_length
    
    def __getitem__(self, idx):
        x = self.features[idx:idx+self.seq_length]
        y = self.labels[idx+self.seq_length]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.long)


In [10]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, shuffle=False)

In [11]:
train_dataset = ForexDataset(train_data, SEQ_LENGTH)
test_dataset = ForexDataset(test_data, SEQ_LENGTH)

In [12]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [17]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_layers, num_heads, seq_length, dropout):
        super(TransformerModel, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads, dropout=dropout)
        self.transformer = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(input_dim * seq_length, 3)
    
    def forward(self, x):
        x = self.transformer(x)  # Output shape: (seq_length, batch_size, input_dim)
        x = x.permute(1, 0, 2)  # Change to (batch_size, seq_length, input_dim)
        x = x.flatten(start_dim=1)  # Flatten to (batch_size, seq_length * input_dim)
        x = self.fc(x)  # Output shape: (batch_size, 3)
        return x


In [18]:
model = TransformerModel(input_dim, NUM_LAYERS, NUM_HEADS, SEQ_LENGTH, DROPOUT).to(device)

c:\Program Files\Python312\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [19]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [20]:
# Training and Evaluation
for epoch in range(EPOCHS):
    model.train()
    train_losses = []
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        x = x.permute(1, 0, 2)  # Seq_Length, Batch_Size, Input_Dim
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            x = x.permute(1, 0, 2)
            outputs = model(x)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {np.mean(train_losses):.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
